In [1]:
!ls data

delme.json
dir.txt
gPlusPlaceId_4_duplicates.json
gPlusPlaceId_filter.json
places.clean.json
places.json
places_gps.json
places_gps_no_dup_indexed.json
places_revs_count.json
places_sub.json
places_sub_gps.json
reviews.clean.json
reviews.json
reviews_gPlusPlaceId_index_no_duplicates.json
reviews_sub.json
users.clean.json
users.json
users_sub.json


In [2]:
import beepy
beepy.beep(sound=1) # 1 : 'coin'
# beepy.beep(sound=5) # 5 : 'ready' too noisy
# beepy.beep(sound=4) # 4 : 'ping'

In [4]:
import numpy as np
import pandas as pd

In [5]:
%%time
df_places_gps = pd.read_json('data/places_gps_no_dup_indexed.json')
df_places_gps.head()
df_places_gps.info()
beepy.beep(sound=4) # 4 : 'ping'

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3061988 entries, 2003-03-06 23:49:04 to 2006-05-05 03:09:30
Data columns (total 10 columns):
 #   Column              Dtype  
---  ------              -----  
 0   name                object 
 1   price               object 
 2   address             object 
 3   hours               object 
 4   phone               object 
 5   closed              bool   
 6   gPlusPlaceId        float64
 7   Y_NS_latitudes      float64
 8   X_EW_longitudes     float64
 9   gPlusPlaceId_index  int64  
dtypes: bool(1), float64(3), int64(1), object(5)
memory usage: 236.5+ MB
CPU times: user 42.8 s, sys: 5.45 s, total: 48.3 s
Wall time: 54.1 s


In [6]:
df_places_gps.head()

,name,price,address,hours,phone,closed,gPlusPlaceId,Y_NS_latitudes,X_EW_longitudes,gPlusPlaceId_index
2003-03-06 23:49:04,Diamond Valley Lake Marina,None,"[2615 Angler Ave, Hemet, CA 92545]","[[Monday, [['6:30 am--4:15 pm']]], [Tuesday, [...",(951) 926-7201,False,1.046995e+20,33.703804,-117.003209,1046994544
2002-08-28 14:26:29,Blue Ribbon Cleaners,None,"[Parole, Annapolis, MD]",None,(410) 266-6123,False,1.030545e+20,38.979759,-76.547538,1030544789
2004-10-18 12:35:01,Portofino,None,"[ул. Тутаева, 1, Nazran, Ingushetia, Russia, 3...","[[Monday, [['9:30 am--9:00 pm']]], [Tuesday, [...",8 (963) 173-38-38,False,1.098103e+20,43.227760,44.762726,1098102901
2001-10-16 22:32:11,T C's Referee Sports Bar,$$,"[5322 W 26th St, Sioux Falls, SD 57106]","[[Monday, [['11:00 am--10:00 pm']]], [Tuesday,...",(605) 361-2208,False,1.003272e+20,43.529494,-96.792244,1003271531
2002-10-03 22:41:13,Carrefour - Palembang Square,None,"[Jl. Angkatan 45, Kompleks Palembang Square, S...","[[Monday, [['8:00 am--9:00 pm']]], [Tuesday, [...",(0711) 359918,False,1.033685e+20,-2.976256,104.742662,1033684873


In [7]:
# Create list of gPlusPlaceId_index column for gps box
# From Alexey.ipynb

# https://en.wikipedia.org/wiki/Earth_radius
# R_avg = 3,959 miles (3950~3963)
# GPS local search subsample
# given local coordinates, and radius, subsample df_places
local = [47.85408756055844, -122.21958066838837] # Alexey's local USPS GPS from google maps 98012
# [latitude North-South, longitude East-West]; opposite order x(EW), y(NS) for geopandas
radius = 5 # miles

import math  # for # math.pi, math.cos()

def check_gps(gps_list, radius):
    '''
    gps_list = [NS_gps, EW_gps] in decimal degrees (Google map right-click output)
    exampe: [47.85408756055844, -122.21958066838837] - 98012 USPS
    radius - desired radius of interest im miles
    
    https://en.wikipedia.org/wiki/Earth_radius
    R_avg = 3,959 miles (3950~3963)

    gps column in places.gps is in the form ['NS_latitudes','EW_longitudes']
    Valid gps coordinates are NS +- 90.0, EW +- 180.0
    There are incorrect NS 75755 (2.45%), EW 75369 (2.44%) of all data
    let's do square first
    IGNORE (BUG) flip through poles EW at exactly +-90.0, and +- at exactly 180.0
    R_avg = 3959 miles, 1/4 equator-> pole 2*PI*R/4 = 90.0, or 
    PI*R/(180) = 1.0 in NS in miles
    
    Y_miles_NS_one_WAY_in_GPS_degrees = Y * 180/(PI * R_earth)
    
    R_ew = R_earth * cos(PI/180*NS_gps); R_earth at NS_gps = 0, 0 at NS_gps = +- 90.0
    
    X_miles_EW_one_WAY_in_GPS_degrees = X * 180/(PI * R_ew)
    
    import math
    math.pi
    3.141592653589793

    '''
    PI = math.pi  # 3.141592653589793
    R_earth = 3959  # Average Earth radius
    # Not GPS dependent going NS
    Y_miles_NS_one_WAY_in_GPS_degrees = radius * 180/(PI * R_earth)
    # GPS dependent on NS_gps - smaller EW circle
    R_ew = R_earth * math.cos(PI/180*gps_list[0])
    X_miles_EW_one_WAY_in_GPS_degrees = radius * 180/(PI * R_ew)
    
    return Y_miles_NS_one_WAY_in_GPS_degrees, X_miles_EW_one_WAY_in_GPS_degrees


In [9]:
# example of usage
Y_NS_deg, X_EW_deg = check_gps([47.85408756055844, -122.21958066838837], 1.0)  # Google map 1.609344 km

In [13]:
# create subsample using current gps, save as subsample db for the model 
# use def 
curr_gps = [47.85408756055844, -122.21958066838837]
radius = 50.0
Y_NS_deg, X_EW_deg = check_gps(curr_gps, radius)  # Google map 1.609344 km
Y_NS_deg, X_EW_deg
# construct a filter based on given GPS +- box
# AND notation as & https://pythoninoffice.com/filter-a-pandas-dataframe-or-and-not/
gps_box_filter = ((df_places_gps.Y_NS_latitudes < curr_gps[0] + Y_NS_deg) & \
           (df_places_gps.Y_NS_latitudes > curr_gps[0] - Y_NS_deg)) & \
          ((df_places_gps.X_EW_longitudes < curr_gps[1] + X_EW_deg) & \
           (df_places_gps.X_EW_longitudes > curr_gps[1] - X_EW_deg))

In [14]:
gps_box_filter.sum() # 23780 places records found at 50 miles radius of zip 98012

23780

In [15]:
gps_box = df_places_gps[gps_box_filter]
gps_box.head() # Seattle, Kirkland, Auburn WA

,name,price,address,hours,phone,closed,gPlusPlaceId,Y_NS_latitudes,X_EW_longitudes,gPlusPlaceId_index
2007-02-10 14:28:17,Sumner's Topgun Muffler Brake,None,"[15608 Main St E, Sumner, WA 98390]",None,(253) 863-3011,False,1.171118e+20,47.202430,-122.221043,1171117697
2004-02-26 09:23:23,Walt's Auto Care Center,None,"[1702 SW Roxbury St, Seattle, WA 98106]","[[Monday, [['8:00 am--5:30 pm']]], [Tuesday, [...",(206) 764-0108,True,1.077787e+20,47.517611,-122.356875,1077787403
2002-07-15 04:57:31,Bishop's Alehouse,None,"[11848 98th Pl NE, Kirkland, WA 98034]","[[Monday, [['3:00 pm--2:00 am']]], [Tuesday, [...",(425) 629-3742,False,1.026709e+20,47.706644,-122.210230,1026709051
2006-10-02 16:41:47,Auburn Avenue Dental Clinic: Walia Jyotsana MD,None,"[130 3rd St NE, Auburn, WA 98002]",None,(253) 833-3680,True,1.159807e+20,47.310139,-122.227524,1159807307
2003-10-15 23:21:29,Community Bible Fellowship,None,"[11227 Renton Ave S, Seattle, WA 98178]",None,(206) 772-1620,False,1.066260e+20,47.502029,-122.250980,1066260089


In [16]:
# extract index similar to duplicates
gps_box['gPlusPlaceId_index'] # gPlusPlaceId_index values to use to filter reviews

2007-02-10 14:28:17    1171117697
2004-02-26 09:23:23    1077787403
2002-07-15 04:57:31    1026709051
2006-10-02 16:41:47    1159807307
2003-10-15 23:21:29    1066260089
                          ...    
2006-04-19 10:18:22    1145441902
2002-02-15 09:59:07    1013767147
2007-01-12 23:53:09    1168645989
2002-04-09 15:20:21    1018365621
2003-04-29 07:07:23    1051600043
Name: gPlusPlaceId_index, Length: 23780, dtype: int64

In [19]:
type(gps_box['gPlusPlaceId_index'])

pandas.core.series.Series

In [21]:
# gPlusPlaceId_index_gps_box = pd.read_json('data/gPlusPlaceId_4_gps_box.json')
gps_box['gPlusPlaceId_index'].to_json('data/gPlusPlaceId_4_gps_box.json', index=True)

In [23]:
gps_box.to_json('data/places_gps_box.json')

In [24]:
!ls data


delme.json
dir.txt
gPlusPlaceId_4_duplicates.json
gPlusPlaceId_4_gps_box.json
gPlusPlaceId_filter.json
places.clean.json
places.json
places_gps.json
places_gps_box.json
places_gps_no_dup_indexed.json
places_revs_count.json
places_sub.json
places_sub_gps.json
reviews.clean.json
reviews.json
reviews_gPlusPlaceId_index_no_duplicates.json
reviews_sub.json
users.clean.json
users.json
users_sub.json


In [ ]:
# Eiffel Tower (real, not in Australia) index = 1112632
df_places_gps.loc[1112632] # !!! loc[1112632], NOT iloc[1112632]
# gPlusPlaceId 118197036100000006144.0
# NOW check against df_reviews 

In [ ]:
# review from TEMP_3 reviews.json
# df.iloc[359117] French review of Eiffel Tower  query 1662 reviews # 1
# gPlusPlaceId 118197036100848205824.0 vs
# gPlusPlaceId 118197036100000006144.0 from places_gps.json
# how to extract nice format from places_gps.json df.loc[~]
# 12 left symbols/digits match
#  Now check 2nd most reviewed Kaaba

In [ ]:
conv_coeff = 1e11
duplicate_1 = 118197036100848205824.0
duplicate_2 = 118197036100000006144.0
print(duplicate_1/conv_coeff, duplicate_2/conv_coeff)
# int() rounds down to 0, round to the nearest integer; 
# The great question is would round mess up say Eiffel Tower paces/reviews match
print(round(duplicate_1/conv_coeff), round(duplicate_2/conv_coeff)) # different 1116019360 1116019359

In [ ]:
# Kaaba query 982 reviews
# df_places_gps.loc[1470841] # The real Kaaba 1470841
# gPlusPlaceId 113708963000000004096.0
# gPlusPlaceId 113708963026857000960.0 from places_gps.json
# how to extract nice format from places_gps.json df.loc[~]
# 10 left symbols/digits match


In [ ]:
conv_coeff = 1e11
duplicate_1 = 113708963000000004096.0
duplicate_2 = 113708963026857000960.0
print(duplicate_1/conv_coeff, duplicate_2/conv_coeff)
# int() rounds down to 0, round to the nearest integer; 
# The great question is would round mess up say Eiffel Tower paces/reviews match
print(round(duplicate_1/conv_coeff), round(duplicate_2/conv_coeff)) # different 1116019360 1116019359

In [ ]:
# Santiago Bernabéu Stadium Madrid 790 reviews
# df_places_gps.loc[1470841] # 
# gPlusPlaceId 107_836_749_011706691584.0 df.loc[602954] from reviews
# gPlusPlaceId 107_836_748_999999995904.0 from df_places_gps.loc[2368404]
# how to extract nice format from places_gps.json df.loc[~]
# 8 left symbols/digits match



In [ ]:
conv_coeff = 1e11
duplicate_1 = 107_836_749_011706691584.0
duplicate_2 = 107_836_748_999999995904.0
print(duplicate_1/conv_coeff, duplicate_2/conv_coeff)
# int() rounds down to 0, round to the nearest integer; 
# The great question is would round mess up say Eiffel Tower paces/reviews match
print(round(duplicate_1/conv_coeff), round(duplicate_2/conv_coeff)) # different 1116019360 1116019359

In [ ]:
# Santiago Bernabéu Stadium Madrid 790 reviews
# gPlusPlaceId 107_836_749_011706691584.0 df.loc[602954] from reviews
# gPlusPlaceId 107_836_748_999999995904.0 from df_places_gps.loc[2368404]
# int(1.99999999999) = 1, but 107836748999999995904.0/1e11 = 1078367490.0
z = int(107836748999999995904.0/1e11)  # from df_places_gps.loc[2368404
z # 107_836_749_0

In [ ]:
# Kaaba
# 113708963000000004096.0  /1e11 ~ 1137089630.0
# 113708963026857000960.0 /1e11 ~  1137089630.26857
# 113708963026857000960.0/1e11
113708963026857000960.0/1e10

In [ ]:
# Taj Mahal 790 reviews
# df_places_gps.loc[220396] # 1e11 box
# gPlusPlaceId 114_299_893_800000012288.0  df_places_gps.loc[220396]
# gPlusPlaceId 114_299_893_842663636992.0 from places_gps.json df.loc[789985]
# how to extract nice format from places_gps.json df.loc[~]
# 10 left symbols/digits match



In [ ]:
conv_coeff = 1e11
duplicate_1 = 114_299_893_800000012288.0
duplicate_2 = 14_299_893_842663636992.0
print(duplicate_1/conv_coeff, duplicate_2/conv_coeff)
# int() rounds down to 0, round to the nearest integer; 
# The great question is would round mess up say Eiffel Tower paces/reviews match
print(round(duplicate_1/conv_coeff), round(duplicate_2/conv_coeff)) # different 1116019360 1116019359

In [ ]:
# # Add column as a string of gPlusPlaceId
# df_places_gps['tmp'] = f"{df_places_gps['gPlusPlaceId']: .0f}"

In [ ]:
df_places_gps.head()

In [ ]:
# !!!!!!!!!!!!!! int() rounds to 0 => duplicates, round() to the nearest int
# df_places_gps['gPlusPlaceId_index'] = df_places_gps['gPlusPlaceId'].transform(lambda x: int(x/1e11))
df_places_gps['gPlusPlaceId_index'] = df_places_gps['gPlusPlaceId'].transform(lambda x: round(x/1e11))
df_places_gps.head()

In [ ]:
df_places_gps.set_index('gPlusPlaceId_index', inplace=True)

In [ ]:
# Eiffel Tower
# df_places_gps.loc[1181970361] # use loc - actual index, not sequential
# Kaaba
# df_places_gps.loc[1137089630] # use loc - actual index, not sequential
# Santiago Bernabéu Stadium
# df_places_gps.loc[107_836_749_0] # use loc - actual index, not sequential
# 
df_places_gps.loc[107_836_749_0] # use loc - actual index, not sequential

In [ ]:
!ls data

In [ ]:
whos

In [ ]:
backup = df_places_gps.copy(deep=True)

In [ ]:
df_places_gps['gPlusPlaceId_index'] = df_places_gps.index

In [ ]:
df_places_gps.head()

In [ ]:
# boolean = not df_places_gps["gPlusPlaceId_index"].is_unique      # True (credit to @Carsten)
boolean = df_places_gps["gPlusPlaceId_index"].duplicated().any()
boolean

In [ ]:
conv_coeff = 1e11
duplicate_1 = 1.1160193599999998e+20
duplicate_2 = 1.1160193590000001e+20
print(duplicate_1/conv_coeff, duplicate_2/conv_coeff)
# int() rounds down to 0, round to the nearest integer; 
# The great question is would round mess up say EIffel Tower paces/reviews match
print(round(duplicate_1/conv_coeff), round(duplicate_2/conv_coeff)) # different 1116019360 1116019359

In [ ]:
# iloc[0] gPlusPlaceId 104991888599999987712.0 name: Osteria "Libertaria", address ['Calata Matteotti, 12', '57037 Portoferraio Livorno', 'Italy']
# iloc[1] gPlusPlaceId 104991888599999987712.0 name Kiosk am ZOB Reinhold Schöttke, address [Mölln, Germany]
df_places_gps.loc[1049918886].iloc[0]['address']

In [ ]:
df_places_gps.shape # (3087402, 10)
#  duplicates [25414 rows x 10 columns]
25_414*100/3_087_402  # 0.8231516336388977 0.8% with messed up gPlusPlaceId's


In [ ]:
df_places_gps.duplicated(subset=['gPlusPlaceId_index']).sum() # 25414 duplicate indices
# How many places overall ?

In [ ]:
duplicate_in_gPlusPlaceId_index.shape

In [ ]:
duplicate_Ids = duplicate_Ids.to_frame()
duplicate_Ids['gPlusPlaceId_index'] = duplicate_Ids.index
# duplicate_Ids.reindex()
# duplicate_Ids.columns()
duplicate_Ids.head()

In [ ]:
duplicate_Ids.head()

In [ ]:
duplicate_Ids = duplicate_in_gPlusPlaceId_index[duplicate_in_gPlusPlaceId_index == True]
duplicate_Ids.shape # (25414,) pandas.core.series.Series duplicate ids
#  are there duplicates in duplicates ?!

duplicate_Ids = duplicate_Ids.to_frame()
duplicate_Ids['gPlusPlaceId_index'] = duplicate_Ids.index

# Approach 2: First store boolean array, check then remove
DD_index = duplicate_Ids.duplicated(subset=['gPlusPlaceId_index'])
if DD_index.any():
    print(duplicate_Ids.loc[DD_index], end='\n\n')

In [ ]:
 # len() = 25278-25414 = -136(some appered several times)
unique_duplicates_gPlusPlaceId_index = pd.DataFrame(
    pd.unique(duplicate_Ids['gPlusPlaceId_index']))
unique_duplicates_gPlusPlaceId_index.rename(columns = {'0':'gPlusPlaceId_index'}, inplace = True)
unique_duplicates_gPlusPlaceId_index.head()
unique_duplicates_gPlusPlaceId_index.to_json('data/gPlusPlaceId_4_duplicates.json')

In [ ]:
z=pd.read_json('data/gPlusPlaceId_4_duplicates.json')
z.head(), z.shape
z.loc[25277]

In [ ]:
type(unique_duplicates_gPlusPlaceId_index)

In [ ]:
# type(df_places_gps.loc[duplicate_in_gPlusPlaceId_index])
duplicate_Ids = duplicate_in_gPlusPlaceId_index[duplicate_in_gPlusPlaceId_index == True]
duplicate_Ids.shape # (25414,) pandas.core.series.Series duplicate ids
duplicate_Ids.to_json('data/gPlusPlaceId_4_duplicates.json')

In [ ]:
df_places_gps.loc[~duplicate_in_gPlusPlaceId_index].shape # (25414, 10) duplicates 0.82 %; ~(NOT dupl)(3061988, 10)

In [ ]:
# Approach 2: First store boolean array, check then remove
duplicate_in_gPlusPlaceId_index = df_places_gps.duplicated(subset=['gPlusPlaceId_index'])
if duplicate_in_gPlusPlaceId_index.any():
    print(df_places_gps.loc[duplicate_in_gPlusPlaceId_index], end='\n\n')


In [ ]:
# iloc[0] 104991888599999987712.0
# iloc[1] 104991888599999987712.0
df_places_gps.loc[1049918886].iloc[1]

In [ ]:
%%time
# Attempt to save non-duplicates with corrected gPlusPlaceId_index from int(to 0) to round (to nearest)
df_places_gps.loc[~duplicate_in_gPlusPlaceId_index].to_json('data/places_gps_no_dup_indexed.json', index=True)
beepy.beep(sound=4) # 4 : 'ping'

In [ ]:
%%time
read_places_indexed = pd.read_json('data/places_gps_no_dup_indexed.json')
read_places_indexed.set_index('gPlusPlaceId_index', inplace=True)
beepy.beep(sound=4) # 4 : 'ping'
read_places_indexed.head()

In [ ]:
junk = pd.DataFrame({'A': range(3), 'B': range(1, 4)})
junk

In [ ]:
junk['C'] = junk['A'].transform(lambda x: int(100*x/5.3))
junk

In [ ]:
df_places_gps = df_places_gps.set_index(['gPlusPlaceId'])

In [ ]:
%%time
# search_list = df_places_gps[df_places_gps['address'].str.contains("Madrid")]
search_list = df_places_gps[df_places_gps['name'].str.contains("Taj Mahal")] #df
len(search_list)
# beepy.beep(sound=4) # 4 : 'ping'

In [ ]:
search_list[:5]

In [ ]:
df_places_gps.loc[search_list[:5]]

In [ ]:
%%time
search_list = df_places_gps[df_places_gps['address'].str.contains("Madrid")]
beepy.beep(sound=4) # 4 : 'ping'
print(f'len(search_list) = {len(search_list)}')

In [ ]:
# df_places_gps[df_places_gps['name'].isin(['Eiffel Tower'])].index
search_list = df_places_gps[df_places_gps['name'].isin(['Stadium'])].index.to_list()

In [ ]:
# df_places_gps.loc[118197036100848205824.0]
df_places_gps.loc[1.181970361e+20]  #  Eiffel Tower

In [ ]:
# df_places_gps.loc[[169796, 1470841, 1826505]] # The real Kaaba 1470841
# df_places_gps.loc[1470841] # The real Kaaba 1470841
df_places_gps.loc[search_list] # The real Kaaba 1470841

In [ ]:
df_places_gps.loc[1.0469945439999998e+20]

In [ ]:
# df_places_gps.loc[118197036100848205824.0]

In [ ]:
#  from reviews len('118197036100848205824') Eiffel tower TEMP3
df_places_gps['gPlusPlaceId']

In [ ]:
tmp = df_places_gps['gPlusPlaceId'].iloc[0]
tmp = format(tmp, 'f') # '104699454399999983616.000000'
tmp

In [ ]:
tmp = df_places_gps['gPlusPlaceId'].iloc[0]
tmp=f"{tmp:.0f}"
tmp # ============================================== can it be done on the whole series/column?
len(tmp), tmp  # 21 characters

In [ ]:
len('1.181970361008482') # 17 characters Eiffel Tower from TEMP3 

In [ ]:
tmp = df_places_gps['gPlusPlaceId'].iloc[0]
tmp = repr(tmp) # '1.0469945439999998e+20'
tmp

In [ ]:
tmp = df_places_gps['gPlusPlaceId'].iloc[0]
tmp = np.format_float_positional(tmp) # '104699454399999980000.'
tmp

In [ ]:
%%time
df_places = pd.DataFrame(open('data/places.clean.json'))
df_places = df_places[0].apply(lambda x: pd.Series(eval(x)))
df_places.head()

In [ ]:
df_places.head()

In [ ]:
df_places.info()

In [ ]:
df_columns_list = df_places.columns.to_list()
type(df_columns_list), df_columns_list

In [ ]:
tmp = df_places.loc[:2].copy(deep=True)
tmp

In [ ]:
tmp = tmp[0].apply(lambda x: pd.Series(eval(x)))
tmp.head()

In [ ]:
tmp.info()

In [ ]:
df_places.loc[:2].to_json('data/delme.json')

In [ ]:
tmp_df = pd.read_json('data/delme.json')
tmp_df.info()

In [ ]:
!ls data

In [ ]:
%%time
df=pd.read_json('data/places_gps.json')  # Wall time: 2min 25s Wall time: 5min 17s
df.head()

In [ ]:
backup = df.copy(deep=True)

In [ ]:
df['junk']=""
df.head()

In [ ]:
gPlusPlaceId_filter = df['gPlusPlaceId'].loc[:1].to_list()
gPlusPlaceId_filter

In [ ]:
whos

In [ ]:
df.loc[df['gPlusPlaceId'].isin(gPlusPlaceId_filter), 'junk'] = 7
df.head()

In [ ]:
%%time
places_revs_count=pd.read_json('data/places_revs_count.json', typ='series')
places_revs_count
# use index (should have saved name "gPlusPlaceId")

In [ ]:
places_revs_count.index

In [ ]:
whos

In [ ]:
df.head()

In [ ]:
#  3rd most reviewed place 790 ratings
id_box = 1e11
gPlusPlaceId =  107836749011706691584.0 # from reviews
# gPlusPlaceId  107836748999999995904.0 # from places
(df_places_gps.loc[(df_places_gps['gPlusPlaceId']>=gPlusPlaceId - id_box) & 
        (df_places_gps['gPlusPlaceId']<=gPlusPlaceId + id_box),:])

In [ ]:
df_places_gps.loc[2368404]

In [ ]:
len('1181970361008482')

In [ ]:
df['gPlusPlaceId'].isin(places_revs_count.index)

In [ ]:
filter_have_ratings = df.loc[df['gPlusPlaceId'].isin(places_revs_count.index), 'gPlusPlaceId']
filter_have_ratings

In [ ]:
gPlusPlaceId_filter.head()

In [ ]:
df = backup.copy(deep=True)
df.head()

In [ ]:
df.info()

In [ ]:
gPlusPlaceId_filter.info()

In [ ]:
# https://developers.google.com/maps/documentation/places/web-service/place-id
# A place ID is a textual identifier that uniquely identifies a place. 
# The length of the identifier may vary (there is no maximum length for Place IDs). Examples:

# ChIJgUbEo8cfqokR5lP9_Wh_DaM
# GhIJQWDl0CIeQUARxks3icF8U8A
# EicxMyBNYXJrZXQgU3QsIFdpbG1pbmd0b24sIE5DIDI4NDAxLCBVU0EiGhIYChQKEgnRTo6ixx-qiRHo_bbmkCm7ZRAN
# EicxMyBNYXJrZXQgU3QsIFdpbG1pbmd0b24sIE5DIDI4NDAxLCBVU0E
# IhoSGAoUChIJ0U6OoscfqokR6P225pApu2UQDQ



# gPlusPlaceId_filter['gPlusPlaceId'].to_list() # type: float
gPlusPlaceId_filter['gPlusPlaceId'] = gPlusPlaceId_filter['gPlusPlaceId'].astype(int)
gPlusPlaceId_filter.head()
# [1.083788035e+20,
#  1.0069647129999999e+20,
#  1.154916021e+20,
#  1.1421892040000001e+20,
#  1.0968482829999999e+20,
#  1.0080166800000002e+20,
#  1.1671256190000002e+20,
#  1.092820277e+20,
#  1.114975692e+20,
#  1.01969087e+20,
#  1.084170551e+20,
#  1.043224385e+20,
#  1.1130349049999999e+20,
#  1.09901333e+20,

In [ ]:
backup.head()

In [ ]:
# tmp_filter = df['gPlusPlaceId'] > gPlusPlaceId_filter['gPlusPlaceId'].to_list()[0] # > 10 works 11,453,845 all True
# tmp_filter
gps_filter = df['gPlusPlaceId'].isin(gPlusPlaceId_filter['gPlusPlaceId'].to_list()[0:2])
# df[gps_filter]
gps_filter.unique()

In [ ]:
tmp_filter = df[df['gPlusPlaceId'] = 10] # gPlusPlaceId_filter.iloc[0]
tmp_filter

In [ ]:
gPlusPlaceId_filter.iloc[0]

In [ ]:
df.gPlusPlaceId.loc[gPlusPlaceId_filter.gPlusPlaceId].head()

In [ ]:
df[gps_filter].head()

In [ ]:
gps_filter

In [ ]:
df.head()